In [41]:
from google.cloud import speech
from google.cloud import texttospeech
import os

In [42]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "jai-dev-mlconsole-poc.json"

In [43]:
speech_client = speech.SpeechClient()

In [44]:
# file_path = "../mono.wav"
file_path = "../audio.wav"
with open(file_path, 'rb') as audio:
    data=audio.read()

In [45]:
audio = speech.RecognitionAudio(content=data)
config = speech.RecognitionConfig(
    # encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=44100,
    language_code="en-US",
)

In [46]:
response = speech_client.recognize(config=config, audio=audio)

In [47]:
transcription_list = []
for result in response.results:
    transcription_list.append(result.alternatives[0].transcript)

In [48]:
transcription = " ".join(transcription_list)
print(transcription)

In [36]:
# Instantiates a client
text_client = texttospeech.TextToSpeechClient()

# Set the text input to be synthesized
# synthesis_input = texttospeech.SynthesisInput(text="Hello, World!")
synthesis_input = texttospeech.SynthesisInput(text="new jersey est parfois calme pendant l' automne , et il est neigeux en avril .")

# Build the voice request, select the language code ("en-US") and the ssml
voice gender ("neutral")
voice = texttospeech.VoiceSelectionParams(
    language_code="fr-FR", ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL
)

# Select the type of audio file you want returned
audio_config = texttospeech.AudioConfig(
    audio_encoding=texttospeech.AudioEncoding.MP3
)

# Perform the text-to-speech request on the text input with the selected
# voice parameters and audio file type
response = text_client.synthesize_speech(
    input=synthesis_input, voice=voice, audio_config=audio_config
)

# The response's audio_content is binary.
with open("output.mp3", "wb") as out:
    # Write the response to the output file.
    out.write(response.audio_content)
    print('Audio content written to file "output.mp3"')

Audio content written to file "output.mp3"


In [6]:
import numpy as np

In [8]:
preproc_english_sentences = np.load('preproc_english_sentences.npy')

In [9]:
preproc_english_sentences.shape

(137861, 15)

In [10]:
max_len_eng = preproc_english_sentences.shape[1]

In [26]:
type(max_len_eng)

int

In [12]:
lst = [max_len_eng]

In [13]:
lst

[15]

In [14]:
np.save("max_len_arr.npy", lst)

In [16]:
nparr = np.load("max_len_arr.npy")

In [24]:
nparr.tolist()

[15]

In [27]:
type(nparr[0])

numpy.int64

In [30]:
type(nparr.item())

int

In [43]:
type(preproc_english_sentences.shape[-1])

int

In [155]:
import pickle

In [156]:
sentence='he watch a old yellow car'

In [151]:
with open('english_tokenizer.pickle', 'rb') as handle:
     english_tokenizer = pickle.load(handle)

In [152]:
lst=[]

In [157]:
for word in sentence.split(" "):
    if english_tokenizer.word_index.get(word):
        lst.append(english_tokenizer.word_index.get(word))   

In [159]:
len(lst)

5

In [160]:
def final_predictions(self, model, sentence, french_tokenizer, english_tokenizer, max_len):
        """
           Turn logits from a neural network into text using the tokenizer
           :param sentence: sequence of string or single sentence
           :param french_tokenizer: tokenize form of french dataa set
           :param english_tokenizer:tokenize form of english data set
           :param preproc_english_sentences
           :return: predicted english sentece
        """

        # sentence = [english_tokenizer.word_index[word] for word in sentence.split()]

        lst=[]
        for word in sentence.split(" "):
            if english_tokenizer.word_index.get(word):
                lst.append(english_tokenizer.word_index.get(word))
        print(lst)        
        lst=pad_sequences([lst],maxlen=max_len,padding='post')        
        
        # sentence = pad_sequences([sentence],maxlen=max_len,padding='post')
        lst = model.predict(lst, len(lst))
        # reshaping because after prediction size is (1,21,345)
        lst = lst.reshape(21, 345)

        #object of preprocessing class
        prep=preprocessing()

        res = prep.logits_to_text(lst, french_tokenizer)

        return res